In [38]:
from xml.dom import minidom
import re
import pandas as pd
import sys
import pickle
import numpy as np
import time
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
sys.__stdout__=sys.stdout

In [39]:
f=open('Stopwords.txt')
s_words=f.read()
s_words=s_words.split('\n')
s_words.append('')

pickle_read = open("wordmap_all_600.pickle","rb")
all_words=pickle.load(pickle_read)
pickle_read.close()
all_words=list(all_words.keys())

f=open('verbs.txt','r')
verbs=f.read()
f.close()
verbs=verbs.split('\n')

irr=pd.read_csv('irr_verbs.txt',delimiter=',',names=['root','past','past_par'])
verb_root=list(irr['root'])
past=list(irr['past'])
past_pp=list(irr['past_par'])


def stem_regular_verb(word):
    if len(word)<=3:
        return word
    if word[-2:]=='es':
        if word[:-2] in verbs:
            word=word[:-2]
        elif word[:-2] in all_words:
            word=word[:-2]
    
    elif word[-1:]=='s':
        if word[:-1] in verbs:
            word=word[:-1]
        elif word[:-1] in all_words:
            word=word[:-1]
    
    elif word[-3:]=='ing':
        if word[:-3] in verbs:
            word=word[:-3]
        if word[:-4] in verbs:
            word=word[:-4]
        else:
            word=word[:-3]
    
    elif word[-2:]=='ed':
        if word[:-2] in verbs:
            word=word[:-2]
        elif word[:-1] in verbs:
            word=word[:-1]
    #word=stem_irregular(word)
    return word
def stem_simple(word):
    if word[-2:]=='es':
            word=word[:-2]   
    elif word[-1:]=='s':
            word=word[:-1]
    elif word[-3:]=='ing':
            word=word[:-3]
    
    elif word[-2:]=='ed':
            word=word[:-2]
    return word

def stem_irregular_verb(word):
        if word in past:
            ind=past.index(word)
            word=verb_root[ind]
        elif word in past_pp:
            ind=past_pp.index(word)
            word=verb_root[ind]
        return word
def stem_word(word):
    temp=word
    #word=stem_regular_verb(word)
    #word=stem_irregular_verb(word)
    #word=ps.stem(word)
    if(len(word)<=1):
        return temp
    return word   

In [52]:
def remove_tags(doc):
    text=doc.lower()
    #text=re.sub('\n',' ',text)
    #text=re.sub('-',' ',text)
    #text=re.sub('[()?.@*#&!,$]','',text)
    #text=re.sub('[\']','',text)
    #text=re.sub('[^a-z0-9]',' ',text)
    #words=doc.split(' ')
    text=re.sub('<[^<>]+>',' ',text)
    words=re.findall('\w+',text)
    return words

def make_dictionary(topics,max_doc,root):
    index=0
    wordmap={}
    word_count=[]
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            doc=doc.attributes['Body'].value
            if len(doc)<5:
                continue
            words = remove_tags(doc)
            for word in words:
                if word in s_words or len(word)<=1:
                    continue
                word=stem_word(word)
                if word not in wordmap:
                    wordmap[word]=index
                    word_count.append(int(1))
                    index=index+1
                elif word in wordmap:
                    ind=wordmap[word]
                    word_count[ind]=int(word_count[ind])+1
            doc_count=doc_count+1
            if doc_count>=max_doc:
                break
    return wordmap,word_count
def remove_minor_words(wordmap,word_count,min_occurance):
    words=list(wordmap.keys())
    freq=list(wordmap.values())
    word_df=pd.DataFrame(words,index=list(wordmap.values()),columns=['words'])
    word_df=word_df.sort_index()
    word_df['freq']=wc
    word_df=word_df.sort_values('freq',ascending=False)
    mask=word_df['freq']<=min_occurance
    word_df=word_df[mask]
    
    ss_words=(list(word_df['words']))
    print('initial length of wordmap :',len(wordmap))
    for word in ss_words:
        del wordmap[word]
    
    _wordmap={}
    ind=0
    for word in wordmap.keys():
        _wordmap[word]=ind
        ind+=1
    print('reduced length of wordmap :',len(_wordmap))
    return _wordmap

def vectorize_seq(topics,max_doc,wordmap,root):
    n_row=len(topics)*(max_doc)
    vectors=np.zeros((n_row,len(wordmap)),dtype=np.float64)
    vct_label=[]
    row=0
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:

            #vector = [0]*(len(wordmap)+2)
            doc=doc.attributes['Body'].value
            if len(doc)<5:
                continue
            words = remove_tags(doc)
            for word in words:
                word=stem_word(word)
                if word in wordmap:
                    ind=wordmap[word]
                    vectors[row,ind]=vectors[row,ind]+1
            row=row+1
            vct_label.append(topic)
            word_list.append(words)
            doc_count=doc_count+1
            if doc_count>=max_doc:
                break
    return vectors,vct_label


In [53]:
def eucl_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    for row in range(len(vct_labels)):
        vect=vectors[row]
        dist=np.abs(vect-test_vect)
        dist=np.square(dist)
        dist=np.sum(dist)
        dist=np.sqrt(dist)
        label=vct_labels[row]
        h_dists.append([dist,label])
    return h_dists
def humm_dis(test_vector,vectors,vct_labels):
    h_dists=[]
    
    test_vect=np.array(test_vector>0,dtype=np.float64)
    vects=np.array(vectors>0,dtype=np.float64)
    #test_vect=test_vector
    #vects=vectors
    for row in range(len(vct_labels)):
        vect=vects[row]
        label=vct_labels[row]
        dist=np.abs(vect-test_vect)
        #dist=dist*[test_vect!=0]
        dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
def cos_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    
    vectors_bin= np.array(vectors>0,dtype=np.float64)
    wc_vect=np.sum(vectors_bin,axis=0)
    no_docs=len(vectors)
    
    IDF=no_docs/(wc_vect+1)
    IDF=np.log(IDF)
    
    for row in range(len(vct_labels)):
        train_vect=vectors[row]
        
        train_label=vct_labels[row]
        
        train_total=np.sum(train_vect)
        test_total=np.sum(test_vect)
        
        TF_test=test_vect/test_total
        test_vec=TF_test*IDF
        
        temp=np.square(test_vec)
        temp=np.sum(temp)
        test_vec_mag=np.sqrt(temp)
        
        TF_train=train_vect/train_total
        train_vec=TF_train*IDF
        
        temp=np.square(train_vec)
        temp=np.sum(temp)
        train_vec_mag=np.sqrt(temp)
        
        X=np.sum(test_vec*train_vec)
        Y=train_vec_mag*test_vec_mag
        
        dist=X/Y
        h_dists.append([dist,train_label])
    return h_dists

In [54]:
def knn(nn,test_vect,vectors,vct_labels,method):
    if method=='euc':
        h_dists=eucl_dis(test_vect,vectors,vct_labels)
    if method=='hum':
        h_dists=humm_dis(test_vect,vectors,vct_labels)
    if method=='cos':
        h_dists=cos_dis(test_vect,vectors,vct_labels)
    h_df=pd.DataFrame(h_dists,columns=['dist','label'])
    
    if method=='euc' or method=='hum':
        h_df=h_df.sort_values('dist',ascending=True)
    if method=='cos':
        h_df=h_df.sort_values('dist',ascending=False)
    print(h_df.head(5))
    lb=h_df['label']
    lb=lb[:nn]
    print(lb.values)
    
    if k==1:
        lb=lb.mode()
        lb=lb.values[0]
        
    if k==3:
        if len(lb.unique())==k:
            lb=lb[:1]
        lb=lb.mode()
        lb=lb.values[0]
    if k==5:
        if len(lb.mode())>=2:
            lb=lb[:3]
            if len(lb.mode())>=2:
                lb=lb[:1]
        lb=lb.mode()
        lb=lb.values[0]
        
                        
    return(lb)

In [55]:
root='Dataset/Training/'
root_test='Dataset/Test/'
f=open('Stopwords.txt')
s_words=f.read()
s_words=s_words.split('\n')
s_words.append('')

#topics=['3d_Printer','Coffee','Arduino','Astronomy','Biology']#,'Chess','Cooking','Law','Space','Windows_Phone','Wood_Working']
#topics=['3d_Printer','Coffee','Chess','Astronomy','Law','Space']#,,'Cooking',,,'Windows_Phone','Wood_Working']
#topics=['3d_Printer','Coffee','Arduino','Astronomy','Biology','Chess','Cooking','Law','Space','Windows_Phone','Wood_Working']

#topics=['Anime']
#topics=['3d_Printer','Coffee','Chess']
topics=['3d_Printer','Anime','Arduino']
#topics=['3d_Printer','Coffee','Law']
 
#wordmap=make_dictionary(topics,100,root)
#vectors=vectorize_seq(topics,100,wordmap,root)
word_list=[]
write_backup=1
max_docs_train=200
min_occurance=int(max_docs_train/200)
if write_backup==1:
    print('Making dictionary :')
    wordmap,wc=make_dictionary(topics,max_docs_train,root)
    #print('Removing minor words:')
    #wordmap=remove_minor_words(wordmap,wc,min_occurance)
    print('Vectorizing Sequence:')
    vectors,vct_labels=vectorize_seq(topics,max_docs_train,wordmap,root)
    #pickle_out = open("wordmap_all_200.pickle","wb")
    #pickle.dump(wordmap, pickle_out)
    #pickle_out.close()
    #pickle_out = open("vectors_test.pickle","wb")
    #pickle.dump(vectors, pickle_out)
    #pickle_out.close()

read_backup=0
if read_backup==1:
    print('Reading backup')
    pickle_read = open("wordmap_500.pickle","rb")
    wordmap=pickle.load(pickle_read)
    pickle_read.close()

    pickle_read = open("vectors_500.pickle","rb")
    vectors=pickle.load(pickle_read)
    pickle_read.close()

#words=list(wordmap.keys())
#freq=list(wordmap.values())
#word_df=pd.DataFrame({'words':words,'freq':freq})
#word_df=word_df.sort_values('freq',ascending=False)
    



Making dictionary :
3d_Printer
Anime
Arduino
Vectorizing Sequence:
3d_Printer
Anime
Arduino


In [56]:
print(len(wordmap))
#  7850
#  6978

7387


In [57]:
#words=list(wordmap.keys())
#freq=list(wordmap.values())
#word_df=pd.DataFrame(words,index=list(wordmap.values()))
#word_df=word_df.sort_index()
#word_df['freq']=wc
#word_df=word_df.sort_values('freq',ascending=False)
#print(word_df[0:50])
#print(len(vectors))

In [58]:

max_docs_test=20
test_vector,true_lbs=vectorize_seq(topics,max_docs_test,wordmap,root_test)

n=len(topics)*max_docs_test
methods=['cos','euc','hum']
k_val=[1,3,5]
#k_val=[5]

log=True
no_topics=len(topics)
tot_doc=no_topics*max_docs_train
tot_doc_test=no_topics*max_docs_test

acc_log=''
acc_log+='********************************************************************************************\n'
acc_log+=str(topics)+'\n'
acc_log+='stem : none , re : findall , minor_remove: 3\n'
acc_log+=str(['no_topic','train doc per topic','train_doc','test doc per topic','test_doc','word map'])+'\n'
acc_log+=str([no_topics,max_docs_train,tot_doc,max_docs_test,tot_doc_test,str(len(wordmap))])+'\n'
acc_log+='********************************************************************************************\n'
acc_log+=str(['k','method','accuracy','time'])+'\n'
for k in k_val:
    for method in methods:
        acc=0
        T=time.time()
        for i in range(n):
            true_lb=true_lbs[i]
            test_vect=test_vector[i,:]
            pred_lb=knn(k,test_vect,vectors,vct_labels,method)
            print(pred_lb,true_lb)
            if pred_lb==true_lb:
                acc=acc+1
                print(pred_lb,true_lb,acc,i+1)
        accuracy=acc/(n)
        T=time.time()-T
        print('accuray',accuracy)
        print('Time :',T)
        acc_log+=(str([k,method,accuracy,T])+'\n')

acc_log=re.sub('[\[\] ]','',acc_log)
if log==True:
    file=open('Report/backup_exp.txt','a')
    file.write(acc_log)
    file.close()
    pass

#68.75% eucl 60 82.92
#53.33 humm 60  81.25
#87.08 cos 94.17

3d_Printer
Anime
Arduino
         dist       label
189  0.225432  3d_Printer
193  0.179125  3d_Printer
188  0.159965  3d_Printer
56   0.155992  3d_Printer
69   0.141279  3d_Printer
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 1 1
         dist       label
193  0.426594  3d_Printer
194  0.354216  3d_Printer
72   0.317425  3d_Printer
66   0.285391  3d_Printer
189  0.253207  3d_Printer
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 2
         dist       label
189  0.524929  3d_Printer
3    0.273876  3d_Printer
58   0.168516  3d_Printer
15   0.145811  3d_Printer
188  0.119664  3d_Printer
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 3
         dist       label
168  0.138601  3d_Printer
163  0.125119  3d_Printer
349  0.086176       Anime
444  0.082315     Arduino
572  0.070871     Arduino
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 4 4
         dist       label
187  0.161287  3d_Printer
197  0.144485  3d_Printer
128  0.134534  3d_Pri

         dist    label
545  0.176734  Arduino
365  0.114810    Anime
520  0.113106  Arduino
523  0.112591  Arduino
563  0.107466  Arduino
['Arduino']
Arduino Arduino
Arduino Arduino 38 42
         dist    label
594  0.205950  Arduino
591  0.156240  Arduino
592  0.155670  Arduino
539  0.154167  Arduino
568  0.133749  Arduino
['Arduino']
Arduino Arduino
Arduino Arduino 39 43
         dist       label
475  0.133334     Arduino
525  0.130612     Arduino
494  0.130313     Arduino
173  0.125637  3d_Printer
416  0.084587     Arduino
['Arduino']
Arduino Arduino
Arduino Arduino 40 44
         dist    label
547  0.339039  Arduino
424  0.273653  Arduino
416  0.246858  Arduino
425  0.215391  Arduino
420  0.168674  Arduino
['Arduino']
Arduino Arduino
Arduino Arduino 41 45
         dist       label
413  0.260201     Arduino
161  0.172516  3d_Printer
452  0.135375     Arduino
466  0.135345     Arduino
581  0.124312     Arduino
['Arduino']
Arduino Arduino
Arduino Arduino 42 46
         dist    label
4

          dist       label
238  13.892444       Anime
106  14.491377  3d_Printer
306  14.525839       Anime
355  14.594520       Anime
117  14.662878  3d_Printer
['Anime']
Anime Anime
Anime Anime 16 24
         dist       label
355  8.246211       Anime
378  8.426150       Anime
162  8.426150  3d_Printer
119  8.426150  3d_Printer
571  8.485281     Arduino
['Anime']
Anime Anime
Anime Anime 17 25
          dist  label
365  15.329710  Anime
268  15.362291  Anime
378  15.491933  Anime
225  15.524175  Anime
337  15.524175  Anime
['Anime']
Anime Anime
Anime Anime 18 26
          dist       label
7    10.816654  3d_Printer
355  10.908712       Anime
268  10.954451       Anime
571  11.000000     Arduino
528  11.000000     Arduino
['3d_Printer']
3d_Printer Anime
         dist       label
355  3.872983       Anime
528  4.123106     Arduino
7    4.123106  3d_Printer
378  4.242641       Anime
119  4.242641  3d_Printer
['Anime']
Anime Anime
Anime Anime 19 28
         dist       label
355  3.316625 

     dist       label
117  90.0  3d_Printer
355  90.0       Anime
545  91.0     Arduino
378  91.0       Anime
129  92.0  3d_Printer
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 9
     dist       label
355  31.0       Anime
35   31.0  3d_Printer
162  32.0  3d_Printer
119  32.0  3d_Printer
23   34.0  3d_Printer
['Anime']
Anime 3d_Printer
     dist       label
7    23.0  3d_Printer
101  24.0  3d_Printer
161  24.0  3d_Printer
6    24.0  3d_Printer
355  25.0       Anime
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 11
     dist       label
355  55.0       Anime
162  56.0  3d_Printer
119  56.0  3d_Printer
571  57.0     Arduino
5    57.0  3d_Printer
['Anime']
Anime 3d_Printer
     dist       label
6    92.0  3d_Printer
35   93.0  3d_Printer
7    93.0  3d_Printer
355  93.0       Anime
119  94.0  3d_Printer
['3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 4 13
     dist       label
7    84.0  3d_Printer
27   85.0  3d_Printer
355  86.0       Anime
119  87

     dist       label
355  28.0       Anime
571  30.0     Arduino
35   30.0  3d_Printer
528  30.0     Arduino
27   31.0  3d_Printer
['Anime']
Anime Arduino
accuray 0.5833333333333334
Time : 5.831116199493408
         dist       label
189  0.225432  3d_Printer
193  0.179125  3d_Printer
188  0.159965  3d_Printer
56   0.155992  3d_Printer
69   0.141279  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 1 1
         dist       label
193  0.426594  3d_Printer
194  0.354216  3d_Printer
72   0.317425  3d_Printer
66   0.285391  3d_Printer
189  0.253207  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 2
         dist       label
189  0.524929  3d_Printer
3    0.273876  3d_Printer
58   0.168516  3d_Printer
15   0.145811  3d_Printer
188  0.119664  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 3
         dist       label
168  0.138601  3d_Printer
163  

...

         dist       label
189  0.387533  3d_Printer
193  0.225656  3d_Printer
194  0.213253  3d_Printer
66   0.185070  3d_Printer
56   0.178726  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 15 15
         dist       label
187  0.170827  3d_Printer
55   0.157465  3d_Printer
197  0.136662  3d_Printer
47   0.134627  3d_Printer
128  0.127444  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 16 16
         dist       label
128  0.172095  3d_Printer
133  0.170303  3d_Printer
25   0.157473  3d_Printer
2    0.138950  3d_Printer
47   0.132362  3d_Printer
['3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 17 17
         dist       label
189  0.272390  3d_Printer
3    0.192276  3d_Printer
51   0.126276  3d_Printer
157  0.119678  3d_Printer
182  0.116148  3d_Printer
['3d_Printer' '3d_Printer

         dist    label
506  0.300748  Arduino
592  0.202894  Arduino
563  0.183170  Arduino
508  0.172738  Arduino
523  0.171494  Arduino
['Arduino' 'Arduino' 'Arduino' 'Arduino' 'Arduino']
Arduino Arduino
Arduino Arduino 49 51
         dist    label
530  0.267793  Arduino
592  0.240009  Arduino
529  0.231500  Arduino
596  0.230443  Arduino
462  0.221413  Arduino
['Arduino' 'Arduino' 'Arduino' 'Arduino' 'Arduino']
Arduino Arduino
Arduino Arduino 50 52
         dist    label
418  0.187758  Arduino
569  0.177891  Arduino
419  0.166791  Arduino
586  0.165377  Arduino
589  0.158479  Arduino
['Arduino' 'Arduino' 'Arduino' 'Arduino' 'Arduino']
Arduino Arduino
Arduino Arduino 51 53
         dist    label
557  0.234404  Arduino
437  0.216535  Arduino
594  0.192811  Arduino
575  0.158549  Arduino
593  0.141940  Arduino
['Arduino' 'Arduino' 'Arduino' 'Arduino' 'Arduino']
Arduino Arduino
Arduino Arduino 52 54
         dist       label
521  0.296104     Arduino
84   0.144952  3d_Printer
496  0.144

          dist       label
238  13.892444       Anime
106  14.491377  3d_Printer
306  14.525839       Anime
355  14.594520       Anime
117  14.662878  3d_Printer
['Anime' '3d_Printer' 'Anime' 'Anime' '3d_Printer']
Anime Anime
Anime Anime 23 24
         dist       label
355  8.246211       Anime
378  8.426150       Anime
162  8.426150  3d_Printer
119  8.426150  3d_Printer
571  8.485281     Arduino
['Anime' 'Anime' '3d_Printer' '3d_Printer' 'Arduino']
Anime Anime
Anime Anime 24 25
          dist  label
365  15.329710  Anime
268  15.362291  Anime
378  15.491933  Anime
225  15.524175  Anime
337  15.524175  Anime
['Anime' 'Anime' 'Anime' 'Anime' 'Anime']
Anime Anime
Anime Anime 25 26
          dist       label
7    10.816654  3d_Printer
355  10.908712       Anime
268  10.954451       Anime
571  11.000000     Arduino
528  11.000000     Arduino
['3d_Printer' 'Anime' 'Anime' 'Arduino' 'Arduino']
Anime Anime
Anime Anime 26 27
         dist       label
355  3.872983       Anime
528  4.123106    

         dist    label
528  6.324555  Arduino
355  6.324555    Anime
415  6.324555  Arduino
571  6.324555  Arduino
412  6.480741  Arduino
['Arduino' 'Anime' 'Arduino' 'Arduino' 'Arduino']
Arduino Arduino
Arduino Arduino 50 60
accuray 0.8333333333333334
Time : 4.894683599472046
     dist       label
355  61.0       Anime
23   62.0  3d_Printer
268  62.0       Anime
117  63.0  3d_Printer
5    63.0  3d_Printer
['Anime' '3d_Printer' 'Anime' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 1 1
     dist       label
75   50.0  3d_Printer
355  52.0       Anime
4    52.0  3d_Printer
35   52.0  3d_Printer
27   53.0  3d_Printer
['3d_Printer' 'Anime' '3d_Printer' '3d_Printer' '3d_Printer']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 2
     dist       label
355  72.0       Anime
27   73.0  3d_Printer
35   74.0  3d_Printer
7    74.0  3d_Printer
524  75.0     Arduino
['Anime' '3d_Printer' '3d_Printer' '3d_Printer' 'Arduino']
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 3
    

     dist       label
355  27.0       Anime
378  28.0       Anime
365  28.0       Anime
117  29.0  3d_Printer
162  30.0  3d_Printer
['Anime' 'Anime' 'Anime' '3d_Printer' '3d_Printer']
Anime Anime
Anime Anime 31 38
      dist       label
306  109.0       Anime
117  111.0  3d_Printer
355  111.0       Anime
378  112.0       Anime
268  112.0       Anime
['Anime' '3d_Printer' 'Anime' 'Anime' 'Anime']
Anime Anime
Anime Anime 32 39
     dist       label
355  19.0       Anime
545  22.0     Arduino
365  22.0       Anime
162  22.0  3d_Printer
268  22.0       Anime
['Anime' 'Arduino' 'Anime' '3d_Printer' 'Anime']
Anime Anime
Anime Anime 33 40
     dist       label
355  13.0       Anime
35   15.0  3d_Printer
378  16.0       Anime
539  16.0     Arduino
162  16.0  3d_Printer
['Anime' '3d_Printer' 'Anime' 'Arduino' '3d_Printer']
Anime Arduino
     dist       label
415  33.0     Arduino
355  34.0       Anime
545  35.0     Arduino
528  36.0     Arduino
4    36.0  3d_Printer
['Arduino' 'Anime' 'Arduino'

In [59]:
print(acc_log)

********************************************************************************************
'3d_Printer','Anime','Arduino'
stem:none,re:findall,minor_remove:3
'no_topic','traindocpertopic','train_doc','testdocpertopic','test_doc','wordmap'
3,200,600,20,60,'7387'
********************************************************************************************
'k','method','accuracy','time'
1,'cos',0.9333333333333333,19.126810312271118
1,'euc',0.75,4.6693198680877686
1,'hum',0.5833333333333334,5.831116199493408
3,'cos',0.9666666666666667,18.94967555999756
3,'euc',0.8333333333333334,4.775836706161499
3,'hum',0.7666666666666667,5.964364767074585
5,'cos',0.9666666666666667,18.952943563461304
5,'euc',0.8333333333333334,4.894683599472046
5,'hum',0.7833333333333333,5.943868398666382



In [37]:
print(len(vectors))
print(len(vct_labels))
vects=np.array(vectors>0,dtype=np.uint8)
print(len(vects))
print(n)

600
600
600
60


In [83]:
from matplotlib import pyplot as plt
vct=[vectors[12][:500]]
test=vectors[15][:498]
dis=eucl_dis(test,vct)
print(vct)
print(test)
print(dis[0][0])
x=range(0,500)
xt=range(0,498)
plt.plot(x,vct[0],xt,test)
plt.show

TypeError: eucl_dis() missing 1 required positional argument: 'vct_labels'

In [ ]:
from matplotlib import pyplot as plt
max_docs_test=20
test_vector,true_lbs=vectorize_seq(topics,max_docs_test,wordmap,root_test)
#_test_vector=test_vector[0]
#_vectors=vectors[0]
#test_vect=np.array(_test_vector>0,dtype=np.int8)
#vects=np.array(_vectors>0,dtype=np.int8)
#x=range(len(vects))
#dist=np.abs(test_vect-vects)
#sum_=np.sum(dist)
#print(sum_,dist[:10])

#plt.plot(x,vects)
#plt.show()
#plt.plot(x,test_vect)
#plt.show()
#plt.plot(x,dist)
#plt.show()
h_dists=humm_dis(test_vector[45],vectors,vct_labels)
h_df=pd.DataFrame(h_dists,columns=['dist','label'])
h_df=h_df.sort_values('dist',ascending=True)
print(h_df)

In [ ]:
def eucl_dis(test_vector,vectors):
    h_dists=[]
    for vector in vectors:
        vect=np.array(vector[:-2])
        vect[vect>1]=1
        label=vector[-1]
        test_vect=np.array(test_vector)
        dist=np.abs(vect-test_vect)
        #dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
test_vector=vectors[0][:-2]
vecto=[vectors[0],vectors[1]]
h_dists=cos_dis(test_vector,vecto)
#h_df=pd.DataFrame(h_dists)
#h_df=h_df.sort_values(0)
print(h_dists)
#print((vectors[15]))

In [ ]:
for i in range(len(vectors)):
    vct=vectors[i][:-2]
    if np.sum(vct)==0:
        break
print(i,word_list[185])

In [109]:
raw='I am hasib\n,i "am" a student() so? @i do& my-duties! porp. and' 
#text=re.sub('\n',' ',raw)
text=raw.lower()
text=re.sub('[^a-z0-9]',' ',text)
#text=re.sub('-',' ',text)
#text=re.sub('[\'"]','',text)
#text=re.sub('[()?.@*#&!,$+]','',text)
#print(text.split(' '))

In [27]:
dt=np.ones((2,3))
print(dt)
dts=np.sum(dt,axis=1)
print(dts)

[[1. 1. 1.]
 [1. 1. 1.]]
[3. 3.]
